<a href="https://colab.research.google.com/github/8ahmedanwer8/rcCarIMUDataClassification/blob/main/imudataClassificiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
import pandas  as pd
import numpy as np 
import pickle
import matplotlib.pyplot as plt
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Augmented_dataset.csv')
df.shape


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(9999, 1806)

In [32]:

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Augmented_dataset.csv')
#shape is 9999x1806. 9999 samples of accel and gyro data. each sample is 1806 columns long
#one accel/gyro data is [[x1,x2...,xn],[y1,y2...,yn],[z1,z2...,zn]] where n is equal to 300
#so we have two arrays each with 1 array which has 3 arrays which has 300 data points

#creating a proper column headers, probably there is a better way to do this but below works
col_names = ["ignore", "ignore"]
for i in range(1,301):
  name = "accelX " + str(i)
  col_names.append(name)
for i in range(1,301):
  name = "accelY " + str(i)
  col_names.append(name)
col_names.append("ignore")
for i in range(1,301):
  name = "accelZ " + str(i)
  col_names.append(name)
col_names.append("ignore")
for i in range(1,301):
  name = "gyroX " + str(i)
  col_names.append(name)
for i in range(1,301):
  name = "gyroY " + str(i)
  col_names.append(name)
for i in range(1,301):
  name = "gyroZ " + str(i)
  col_names.append(name)
col_names.append("ignore")
col_names.append("Labels")

df.columns = col_names

for index, row in df.iterrows():#remove poorly formatted rows, about 181 of these exist
  if pd.isna(row[1805]):
    df.drop(index, inplace=True)


df.iloc[:,1805]= df.iloc[:,1805].apply(lambda x:x.replace('}"', '') if pd.isna(x)!= True else x) #remove }" in the true labels column
df.iloc[:,1805]= df.iloc[:,1805].astype(int)#turn the labels from strings to integers (int64 to be specific)

df.iloc[:,1803]= df.iloc[:,1803].apply(lambda x:x.replace(']]),', '') if pd.isna(x)!= True else x) #remove ]]), in gyro{x,y,z} closing bracket
df.iloc[:,902]= df.iloc[:,902].apply(lambda x:x.replace(']]),', '') if pd.isna(x)!= True else x) #remove ]]), in accel{x,y,z} closing bracket
df.iloc[:,904]= df.iloc[:,904].apply(lambda x:x.replace('array([[', '') if pd.isna(x)!= True else x) #remove array([[, in gyro{x,y,z} opening bracket

df.iloc[:,301]= df.iloc[:,301].apply(lambda x:x.replace(']', '') if pd.isna(x)!= True else x) #remove ] from closing bracket of accelX  
df.iloc[:,601]= df.iloc[:,601].apply(lambda x:x.replace(']', '') if pd.isna(x)!= True else x) #remove ] from closing bracket of accelY  
df.iloc[:,302]= df.iloc[:,302].apply(lambda x:x.replace('[', '') if pd.isna(x)!= True else x) #remove [ from opening bracket of accelY
df.iloc[:,1203]= df.iloc[:,1203].apply(lambda x:x.replace(']', '') if pd.isna(x)!= True else x) #remove ] from closing bracket of gyroX
df.iloc[:,1204]= df.iloc[:,1204].apply(lambda x:x.replace('[', '') if pd.isna(x)!= True else x) #remove [ from opening bracket of gyroY
df.iloc[:,1503]= df.iloc[:,1503].apply(lambda x:x.replace(']', '') if pd.isna(x)!= True else x) #remove ] from closing bracket of gyroY
df.iloc[:,1504]= df.iloc[:,1504].apply(lambda x:x.replace('[', '') if pd.isna(x)!= True else x) #remove [ from opening bracket of gyroZ


df = df.drop(df.columns[[0,1,602,903,1804]], axis=1) #remove certain columns ("{'Acc':, array([[,[ for accelZ, 'Gyr': and 'Label':)
df.replace(to_replace=",", value="", regex = True, inplace = True) #remove the comma after each number in the df

# df.to_csv('shouldbegood.csv', index=False)
df.sample(10)


,accelX 1,accelX 2,accelX 3,accelX 4,accelX 5,accelX 6,accelX 7,accelX 8,accelX 9,accelX 10,accelX 11,accelX 12,accelX 13,accelX 14,accelX 15,accelX 16,accelX 17,accelX 18,accelX 19,accelX 20,accelX 21,accelX 22,accelX 23,accelX 24,accelX 25,accelX 26,accelX 27,accelX 28,accelX 29,accelX 30,accelX 31,accelX 32,accelX 33,accelX 34,accelX 35,accelX 36,accelX 37,accelX 38,accelX 39,accelX 40,...,gyroZ 262,gyroZ 263,gyroZ 264,gyroZ 265,gyroZ 266,gyroZ 267,gyroZ 268,gyroZ 269,gyroZ 270,gyroZ 271,gyroZ 272,gyroZ 273,gyroZ 274,gyroZ 275,gyroZ 276,gyroZ 277,gyroZ 278,gyroZ 279,gyroZ 280,gyroZ 281,gyroZ 282,gyroZ 283,gyroZ 284,gyroZ 285,gyroZ 286,gyroZ 287,gyroZ 288,gyroZ 289,gyroZ 290,gyroZ 291,gyroZ 292,gyroZ 293,gyroZ 294,gyroZ 295,gyroZ 296,gyroZ 297,gyroZ 298,gyroZ 299,gyroZ 300,Labels
8789,6.18383076e-03,3.06877768e-02,6.78633837e-02,9.90929888e-02,1.05758930e-01,7.85935535e-02,4.72661830e-02,4.11985682e-02,4.56993710e-02,3.93345585e-02,2.22263465e-02,6.86207775e-04,-1.74321655e-02,-2.65432357e-02,-3.32246099e-02,-4.36839032e-02,-4.66228877e-02,-2.97397540e-02,3.08179750e-04,3.26058533e-02,6.25105031e-02,8.53210916e-02,8.68567455e-02,5.71611620e-02,2.18457546e-02,1.04388694e-02,1.59391050e-02,2.27888852e-02,3.13239346e-02,4.35605242e-02,4.30734079e-02,1.50425438e-02,-1.36033412e-02,-1.07944002e-02,6.10237519e-03,1.01591994e-02,1.64670747e-02,4.68502553e-02,7.69895485e-02,7.39049932e-02,...,-4.20206083e+00,-4.70327587e+00,-4.52350126e+00,-3.40716088e+00,-2.02202385e+00,-1.00789481e+00,-4.67916735e-01,-3.37354770e-01,-3.39694017e-01,-2.68713925e-01,-2.40196471e-01,-3.86210323e-01,-7.08364089e-01,-1.15040436e+00,-1.55337357e+00,-1.74983257e+00,-1.60779902e+00,-1.09945330e+00,-5.08951042e-01,-1.89584401e-01,-5.16468995e-01,-1.61190583e+00,-2.73907718e+00,-3.21141171e+00,-3.12369376e+00,-2.73799831e+00,-2.32006885e+00,-2.04496991e+00,-1.76343228e+00,-1.37593506e+00,-1.22282027e+00,-1.60381774e+00,-2.30180568e+00,-2.99597520e+00,-3.42397349e+00,-3.40676553e+00,-3.03257920e+00,-2.45382073e+00,-1.82289637e+00,3
5736,4.02007859e-02,1.09646564e-01,3.89478267e-02,-3.79946379e-02,1.27199586e-02,2.65626848e-01,5.48040614e-01,6.70265915e-01,6.43736104e-01,5.33346935e-01,4.04215400e-01,3.13560964e-01,2.83400176e-01,3.15508097e-01,3.64216436e-01,3.82194945e-01,3.76092823e-01,3.64553662e-01,3.49592539e-01,3.29898219e-01,3.11619271e-01,3.02223549e-01,3.04701403e-01,3.17656682e-01,3.24614627e-01,3.11116131e-01,2.96928025e-01,3.04501960e-01,3.13661934e-01,2.98743040e-01,2.78316423e-01,2.77904382e-01,2.81427430e-01,2.64858853e-01,2.37820827e-01,2.20598740e-01,2.28339298e-01,2.67800073e-01,3.07785152e-01,3.11149506e-01,...,-1.26527804e+00,-9.75831875e-01,-8.24779537e-01,-7.53272241e-01,-6.69811500e-01,-5.02571814e-01,-2.52014648e-01,4.39803646e-02,2.75961925e-01,3.55715795e-01,2.96399330e-01,1.49087709e-01,2.29505864e-02,-8.14764598e-03,-1.30035166e-02,-7.83310903e-02,-2.60927397e-01,-5.59078200e-01,-8.02134732e-01,-8.44686481e-01,-7.34662270e-01,-5.72623005e-01,-5.02976473e-01,-6.12719103e-01,-7.62126868e-01,-7.97937091e-01,-6.87471204e-01,-4.45800042e-01,-1.64477472e-01,4.46314058e-02,5.74784151e-02,-1.89827096e-01,-5.28986113e-01,-7.71426209e-01,-8.53898080e-01,-7.67521630e-01,-5.98991643e-01,-4.57953479e-01,-4.54052498e-01,2
8869,6.38168795e-02,5.64265831e-02,3.06899226e-02,7.99490300e-03,9.72952938e-03,4.52044743e-02,7.34358272e-02,5.73536451e-02,3.21405972e-02,3.60773809e-02,3.76878519e-02,2.39007172e-03,-3.45229201e-02,-2.94203467e-02,1.17694100e-02,6.58012422e-02,9.40497111e-02,6.82892300e-02,2.99774033e-02,2.67613576e-02,3.78638320e-02,3.13937303e-02,2.13566071e-02,2.88645699e-02,4.04907739e-02,3.68271599e-02,2.28424167e-02,1.03104495e-02,6.09757189e-03,1.39539182e-02,2.95353907e-02,4.57473016e-02,5.50891014e-02,5.07752012e-02,3.07291635e-02,-7.40883822e-04,-1.24161672e-02,2.42508185e-02,6.80202829e-02,6.56209987e-02,...,1.58874612e+00,9.70785592e-01,9.72076233e-01,1.50746798e+00,1.73737944e+00,1.04621974e+00,-1.69413659e-01,-1.26100539e+00,-

In [7]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/processed_Augmented_dataset.csv')
df.iloc[:,1800].value_counts() #there are 5 labels and about 2000 samples for each label so data is well distributed

#create numpy arrays of the data just to test the model at the end 
predX = [] 
predY = []
predY = df['Labels']
predX = df.iloc[0:,0:1800]
predX = np.array(predX)
predY= np.array(predY)




In [17]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical

#creating numpy arrays of the data (X and y) which will be used for the deep neural network
df = df.sample(frac = 1) #shuffle the data
X = []
y = []
y = df['Labels']
X = df.iloc[0:,0:1800]
y = np.array(y)
X = np.array(X)
y = to_categorical(y) #one hot encode the labels


In [23]:
#tensorboard for graphing model stats
import datetime, os
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

#deep neural network
model = Sequential()
model.add(Dense(512, input_shape = X.shape[1:], activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(6,activation = 'softmax')) #6 is the number of output neurons because it should match the shape of y

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
model.fit(X,y, epochs =20, batch_size = 32, validation_split = 0.2, callbacks=[tensorboard_callback]) 



Epoch 1/2
246/246 [==============================] - 6s 19ms/step - loss: 0.7628 - accuracy: 0.8344 - val_loss: 1.1309 - val_accuracy: 0.9114
Epoch 2/2
246/246 [==============================] - 3s 13ms/step - loss: 0.2074 - accuracy: 0.9524 - val_loss: 2.0180 - val_accuracy: 0.9109


In [24]:
import tensorflow
model.save('rcCar.model')
myModel = tensorflow.keras.models.load_model('rcCar.model')

INFO:tensorflow:Assets written to: rcCar.model/assets


In [27]:
pred = myModel.predict([predX])#outputs the probability distribution for each data sample (X1, X2, X3...)
ctr = 0

one = 'acceleration'
two = 'normal driving'
three = 'collision'
four = 'left turn'
five = 'right turn'

for i in range(1,9818):
  answer = np.argmax(pred[i]) #finds the highest probability from each distribution and that is our model's prediction
  
  if answer == 1: message = one
  if answer == 2: message = two
  if answer == 3: message = three
  if answer == 4: message = four
  if answer == 5: message = five

  print(i, answer," ", message, "\t", predY[i])
  if (answer != predY[i]):
    print("found a mistake!")
    ctr = ctr +1
print("\nThe amount of incorrect predictions were ", ctr)
print((ctr/9818)*100, "% were incorrect")





Streaming output truncated to the last 5000 lines.
5129 1   acceleration 	 1
5130 3   collision 	 1
found a mistake!
5131 3   collision 	 1
found a mistake!
5132 1   acceleration 	 1
5133 1   acceleration 	 1
5134 1   acceleration 	 1
5135 1   acceleration 	 1
5136 1   acceleration 	 1
5137 1   acceleration 	 1
5138 1   acceleration 	 1
5139 1   acceleration 	 1
5140 1   acceleration 	 1
5141 1   acceleration 	 1
5142 1   acceleration 	 1
5143 3   collision 	 1
found a mistake!
5144 1   acceleration 	 1
5145 3   collision 	 1
found a mistake!
5146 1   acceleration 	 1
5147 1   acceleration 	 1
5148 1   acceleration 	 1
5149 1   acceleration 	 1
5150 1   acceleration 	 1
5151 1   acceleration 	 1
5152 1   acceleration 	 1
5153 1   acceleration 	 1
5154 1   acceleration 	 1
5155 1   acceleration 	 1
5156 1   acceleration 	 1
5157 1   acceleration 	 1
5158 1   acceleration 	 1
5159 1   acceleration 	 1
5160 1   acceleration 	 1
5161 1   acceleration 	 1
5162 1   acceleration 	 1
5163 1   